# TF-IDF 

In [186]:
import pandas as pd
import numpy as np
from collections import Counter
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn import metrics
from gensim.models import Word2Vec
import nltk
import re
import string

In [187]:
nltk.download('stopwords')
from nltk.corpus import stopwords
stopwords.words('english')
#df0=pd.read_csv("data.csv")
df = pd.read_csv("data_2.csv")
#df0.info()

[nltk_data] Downloading package stopwords to /home/srj/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [188]:
stopwords = nltk.corpus.stopwords.words('english')
ps = nltk.WordNetLemmatizer()

In [189]:
def clean_text(text):
    no_punctuation = ''.join([word.lower() for word in text if word not in string.punctuation])
    tokens = re.split('\W+', text)
    stems = [ps.lemmatize(word) for word in tokens if word not in stopwords] # Remove Stopwords
    return stems

In [190]:
print(sorted(Counter(df['LABEL']).items()))

[(0, 4846), (1, 4893), (2, 4950)]


In [191]:
X=df['TEXT']
Y=df['LABEL']

In [192]:
print(X.shape)
print(Y.shape)

(14689,)
(14689,)


In [193]:
split_index = 6042
X_train, X_test, Y_train, Y_test = train_test_split(X[:split_index], Y[:split_index],random_state=42)

In [194]:
X_train = X[split_index:]
Y_train = Y[split_index:]

In [195]:
vect = TfidfVectorizer(analyzer=clean_text)
vector = vect.fit_transform(X_train)
X_train = pd.DataFrame(vector.toarray())
X_test = pd.DataFrame(vect.transform(X_test).toarray())

In [196]:
print(X_train.head())
print(X_test.head())

   0     1     2     3     4     5     6     7     8     9     ...  5796  \
0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0  ...   0.0   
1   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0  ...   0.0   
2   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0  ...   0.0   
3   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0  ...   0.0   
4   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0  ...   0.0   

   5797  5798  5799  5800  5801  5802  5803  5804  5805  
0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0  
1   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0  
2   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0  
3   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0  
4   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0  

[5 rows x 5806 columns]
       0     1     2     3     4     5     6     7     8     9     ...  5796  \
0  0.000000   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0  ...   0.0   
1  0.00000

In [197]:
#X = [str(x) for x in X] #converting to iterable object before passing to fit transform
#vect = TfidfVectorizer(analyzer=clean_text)
#vector=vect.fit_transform(X)
#X=pd.DataFrame(vector.toarray())

In [198]:
classifier = LogisticRegression(solver='lbfgs', max_iter=1000)
print(X.shape)
print(Y.shape)
classifier.fit(X_train,Y_train)
predicted = classifier.predict(X_test)
pred_prob=classifier.predict_proba(X_test)
print(metrics.accuracy_score(Y_test, predicted))

(14689,)
(14689,)
0.18199867637326275


# Word 2 VEC

In [176]:
sentences = [text.split() for text in X_train]
model = Word2Vec(sentences, vector_size=100, window=5, min_count=1, workers=4)

In [177]:
X_train_w2v = []
for sentence in sentences:
    sentence_vectors = []
    for word in sentence:
        if word in model.wv:
            sentence_vectors.append(model.wv[word])
    if sentence_vectors:
        X_train_w2v.append(np.mean(sentence_vectors, axis=0))
    else:
        X_train_w2v.append(np.zeros(100))

In [178]:
clf = LogisticRegression()
clf.fit(X_train_w2v, Y_train)
X_train_w2v = np.array(X_train_w2v)

/home/srj/.local/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [180]:
x_test_w2v = np.zeros((len(X_test), model.vector_size))

for i, doc in enumerate(X_test):
    # Split the text document into words
    words = doc.split()
    # Compute the average vector of the words in the document
    vecs = []
    for word in words:
        if word in model:
            vecs.append(model[word])
    if len(vecs) > 0:
        avg_vec = np.mean(vecs, axis=0)
    else:
        avg_vec = np.zeros(model.vector_size)
    x_test_w2v[i] = avg_vec


TypeError: argument of type 'Word2Vec' is not iterable

In [172]:
y_pred = clf.predict(x_test_w2v)
acc = np.mean(y_pred == y_test)
print("Accuracy:", acc)

NameError: name 'x_test_w2v' is not defined